In [1]:
from aimsgb import GrainBoundary, Grain
import numpy as np

In [2]:
def read_poscar(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()

    # Read the header
    system_name = lines[0].strip()

    # Read the scaling factor
    scale = float(lines[1].strip())

    # Read the lattice vectors
    lattice_vectors = []
    for i in range(2, 5):
        lattice_vectors.append([float(x) for x in lines[i].split()])
    lattice_vectors = np.array(lattice_vectors) * scale

    # Read the element symbols and counts
    elements = lines[5].split()
    counts = [int(x) for x in lines[6].split()]

    # Read the coordinate style
    coord_style = lines[7].strip().lower()

    # Read the atomic coordinates
    coords = []
    symbols = []
    start = 8
    for i, count in enumerate(counts):
        end = start + count
        symbols.extend([elements[i]] * count)
        for j in range(start, end):
            coords.append([float(x) for x in lines[j].split()[:3]])
        start = end

    # If the coordinate style is fractional, convert to Cartesian
    if coord_style == 'direct' or coord_style == 'cartesian':
        pass
    elif coord_style == 'fractional':
        coords = np.dot(coords, lattice_vectors)
    else:
        raise ValueError('Unsupported coordinate style: {}'.format(coord_style))

    return system_name, lattice_vectors, symbols, coords

def write_cel(filename, system_name, lattice_vectors, symbols, coords):
    with open(filename, 'w') as f:
        f.write('Comment string\n')
        f.write('0 {:.4f} {:.4f} {:.4f} 90.0000 90.0000 90.0000\n'.format(
            lattice_vectors[0][0]/10, lattice_vectors[1][1]/10, lattice_vectors[2][2]/10))
        for symbol, coord in zip(symbols, coords):
            f.write('{} {:.6f} {:.6f} {:.6f} 1.000000 0.005100 0.000000 0.000000 0.000000\n'.format(
                symbol, coord[0], coord[1], coord[2]))
        f.write('*\n')

In [3]:
s_input = Grain.from_mp_id("mp-2998")

gb = GrainBoundary([0, 0, 1], 5, [0, 0, 1], s_input, uc_a=1, uc_b=1)
structure = Grain.stack_grains(gb.grain_a, gb.grain_b, direction=gb.direction)

filename="BTO_[001]5[001]_uc1x_POSCAR"
structure.to(filename=filename)

poscar_file = filename
replaced_filename = filename.replace("_POSCAR", "")
cel_file = f"{replaced_filename}.cel"

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
system_name, lattice_vectors, symbols, coords = read_poscar(poscar_file)

In [25]:
lattice_vectors
lattice_vectors_transpose = np.transpose(lattice_vectors)
lattice_vectors_transpose
new_lattice_vectors = np.matmul(lattice_vectors_transpose, [0, 0, 1])
new_lattice_vectors

array([-6.33670105,  6.33670105,  0.        ])

In [21]:
coords[0]
np.matmul(lattice_vectors_transpose, coords[0])
coords_transformed = []
for i in range(len(coords)):
    coords_transformed.append(np.matmul(lattice_vectors_transpose, coords[i]))
coords_transformed


[array([-5.06936084, -3.80202063, -6.01152245]),
 array([-7.60404126,  3.80202063, -6.01152245]),
 array([-2.53468042,  1.26734021, -6.01152245]),
 array([-10.13872168,  -1.26734021,  -6.01152245]),
 array([-6.33670105e+00,  8.88178420e-16, -6.01152245e+00]),
 array([-5.06936084,  3.80202063, -2.00384082]),
 array([-7.60404126, -3.80202063, -2.00384082]),
 array([-2.53468042, -1.26734021, -2.00384082]),
 array([-10.13872168,   1.26734021,  -2.00384082]),
 array([-6.33670105e+00,  2.22044605e-15, -2.00384082e+00]),
 array([-10.77239179,   0.63367011,  -8.01536327]),
 array([-6.97037116,  1.90101032, -8.01536327]),
 array([-1.90101032e+00, -6.33670105e-01, -9.61536327e-16]),
 array([-3.16835053,  3.16835053, -8.01536327]),
 array([-5.70303095e+00, -1.90101032e+00, -4.80000000e-16]),
 array([-6.97037116, -4.43569074, -8.01536327]),
 array([-3.16835053, -3.16835053, -8.01536327]),
 array([-4.43569074,  0.63367011, -8.01536327]),
 array([-5.70303095e+00,  4.43569074e+00, -8.00000000e-17]),


In [ ]:



write_cel(cel_file, system_name, lattice_vectors, symbols, coords)